# Using the model server to do an inference using REST

In this notebook we are going to use the REST inference endpoint to query our model.

## First, replace the placeholder with the restUrl you got at the previous step from the Model Serving configuration

The port, the model name and the classes definition file are already filled in.

In [ ]:
model_name = 'yolo'
rest_url = 'change_me' # e.g. 'http://modelmesh-serving.<your-namespace>:8008'
infer_url = f'{rest_url}/v2/models/{model_name}/infer'
classes_file = 'coco.yaml'

## Make sure we have the libraries we need
If you want, you can have a look at requirements.txt to see which libraries are used. One of them is PyTorch, as this is the frameworks that was used to train the model. Some parts of this library are still need to do the image preparation before querying the model.

The installation of the libraries can take up to 1mn, please be patient. **There may be Errors or Warnings during this installation, but you can ignore them**.

In [ ]:
!pip install --upgrade pip
!pip install -qr requirements.txt

# Install a CPU-only version of Pytorch
!pip install -q -r requirements-torch-cpu.txt

## Import the libraries and module where we have all helper code for doing the inference

In [ ]:
import re
import cv2
from matplotlib import pyplot as plt
from PIL import Image

from remote_infer_rest import ort_v5

You can also have a look at the file `remote_infer_rest.py` to get more details on the pre and post-processing that will happen with the data before and after sending it to the inference endpoint.

## Now set the parameters for the inference

In [ ]:
# 1. The image you want to analyze
image_path='images/bus.jpg' # You can replace this with an image you upload
#image_path='images/zidane.jpg' 

# 2. Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.4

# 3. Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.6

## Launch the inference and display the result

In [ ]:
infer=ort_v5(image_path, infer_url, conf, iou, 640, classes_file)
img, out, result = infer()
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img)

## Perfect, we can see that the model serving API is working!

You can now get back to the instructions.

## Time it on multiple images!

In [ ]:
import os
import time

folder_path = 'images'

start_time = time.time()

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Get the full path of the file
    file_path = os.path.join(folder_path, filename)

    # Check if the file path is a file (not a directory)
    if os.path.isfile(file_path):
        # Do the inference on the file
        infer=ort_v5(file_path, infer_url, conf, iou, 640, classes_file)
        img, out, result = infer()
        print(f'{result}')

end_time = time.time()
execution_time = end_time - start_time
print(f"Executed in {execution_time:.2f} seconds")